# Intro to PyMC

In [ ]:
from pymc import DiscreteUniform, Exponential, deterministic, Poisson, Uniform
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-talk')

Consider the following dataset, which is a time series of recorded coal mining disasters in the UK from 1851 to 1962. 

Occurrences of disasters in the time series is thought to be derived from a Poisson process with a large rate parameter in the early part of the time series, and from one with a smaller rate in the later part. We are interested in locating the change point in the series, which perhaps is related to changes in mining safety regulations.

We represent our conceptual model formally as a statistical model:


$D_t$: number of disasters in year t. 

$r_t$: the rate parameter of the Poisson distribution of disasters in year $t$.

$s$: The year in which the rate parameter changes (the switchpoint).

$e$: The rate parameter before the switchpoint s.

$l$: The rate parameter after the switchpoint s.

\begin{align}
(D_t |s,e,l) \sim Poisson(r_t)\\
r_t = \begin{cases} e & \text{if } & t< s\\
        l & \text{if} & t \geq s
      \end{cases}
\end{align}

\begin{align}
s \sim Discrete Uniform(t_l, t_h)\\
e \sim Exponential(r_e)\\
l \sim Exponential(r_l)\\
\end{align}

In [ ]:
disasters_array =  np.array([ 4, 5, 4, 0, 1, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6,
                   3, 3, 5, 4, 5, 3, 1, 4, 4, 1, 5, 5, 3, 4, 2, 5,
                   2, 2, 3, 4, 2, 1, 3, 2, 2, 1, 1, 1, 1, 3, 0, 0,
                   1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1,
                   0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2,
                   3, 3, 1, 1, 2, 1, 1, 1, 1, 2, 4, 2, 0, 0, 1, 4,
                   0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])
years= np.linspace(1850, 1960, len(disasters_array))

In [ ]:
plt.plot(disasters_array, 'o')
plt.xlabel('Year')
plt.ylabel('Number of mining disasters')
plt.title('Mining disasters in the UK')
plt.show()


In [ ]:
switchpoint = DiscreteUniform('switchpoint', lower=0, upper=110, doc='Switchpoint[year]')

In [ ]:
early_mean = Exponential('early_mean', beta=1.)
late_mean = Exponential('late_mean', beta=1.)


Next, we define the variable rate, which selects the early rate early_mean for times before switchpoint and the late rate late_mean for times after switchpoint. We create rate using the deterministic decorator, which converts the ordinary Python function rate into a Deterministic object.:

In [ ]:
@deterministic(plot=False)
def rate(s=switchpoint, e=early_mean, l=late_mean):
    ''' Concatenate Poisson means '''
    out = np.empty(len(disasters_array))
    out[:s] = e
    out[s:] = l
    return out


In [ ]:
# Define disasters: in the language of PyMC this is still a random variable, but one that we actually observed:
disasters = Poisson('disasters', mu=rate, value=disasters_array, observed=True)


In [ ]:
# We then use the Markov Chain Monte Carlo sampling method to fit this model. 
from pymc.examples import disaster_model
from pymc import MCMC
M = MCMC(disaster_model)

In [ ]:
M.sample(iter=10000, burn=1000, thin=10)

## Fitting a model? 
Fitting a model means characterizing its posterior distribution somehow. In this case, we are trying to represent the posterior p(s,e,l|D) by a set of joint samples from it. To produce these samples, the MCMC sampler randomly updates the values of switchpoint, early_mean and late_mean according to the Metropolis-Hastings algorithm [Gelman2004] over a specified number of iterations (iter).

As the number of samples grows sufficiently large, the MCMC distributions of switchpoint, early_mean and late_mean converge to their joint stationary distribution. In other words, their values can be considered as random draws from the posterior p(s,e,l|D). PyMC assumes that the burn parameter specifies a sufficiently large number of iterations for the algorithm to converge, so it is up to the user to verify that this is the case (see chapter Model checking and diagnostics). Consecutive values sampled from switchpoint, early_mean and late_mean are always serially dependent, since it is a Markov chain. MCMC often results in strong autocorrelation among samples that can result in imprecise posterior inference. To circumvent this, it is useful to thin the sample by only retaining every k th sample, where k
k is an integer value. This thinning interval is passed to the sampler via the thin argument.

If you are not sure ahead of time what values to choose for the burn and thin parameters, you may want to retain all the MCMC samples, that is to set burn=0 and thin=1, and then discard the burn-in period and thin the samples after examining the traces (the series of samples). See [Gelman2004] for general guidance.



In [ ]:
# Accessing the samples
M.trace('switchpoint')[:5]

In [ ]:
plt.hist(M.trace('late_mean')[:])
plt.show()

In [ ]:
# PyMC has its own plotting  functionality too: 
from pymc.Matplot import plot
plot(M)
plt.show()

The upper left-hand pane of these figures shows the temporal series of the samples from switchpoint, while below is an autocorrelation plot of the samples. The right-hand pane shows a histogram of the trace. The trace is useful for evaluating and diagnosing the algorithm’s performance (see [Gelman1996]), while the histogram is useful for visualizing the posterior.